In [8]:
from IPython.display import display, Image
import IPython.display as ipd
# import fluidsynth
from midi2audio import FluidSynth
import glob
import numpy as np
import pathlib
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import pretty_midi
import tensorflow as tf

from matplotlib import pyplot as plt

# Tomb added
import time
import copy
import random
import sys
from music21 import midi
import subprocess

In [9]:
#Download Chorales
data_dir = pathlib.Path('/Volumes/MAGIC1/CS50/myMusicGen/data/chorales')
if not data_dir.exists():
  tf.keras.utils.get_file(
      'midi',
      origin='https://github.com/jamesrobertlloyd/infinite-bach/tree/master/data/chorales/midi',
      extract=True,
      cache_dir='.', cache_subdir='data',
  )
filenames = glob.glob(str(data_dir/'**/*.mid*'))
print('Number of files:', len(filenames))

Number of files: 498


In [10]:
class UnsupportedMidiFileException(Exception):
  "Unsupported MIDI File"

In [ ]:
class WrongWindowSize(Exception):
    def __init__(self, window_size):
        self.message = f"Window Size must be between 1 and 62!! Given size: {window_size}"
        super().__init__(self.message)

In [11]:
"""Tomb made a pretty useful function"""
def check_pianoroll_dim(pianoroll):
        rows = len(pianoroll)  # This gives the number of rows
        columns = len(pianoroll[0])  # This assumes all rows have the same length

        print("Number of rows:", rows) # represents sequence length
        print("Number of columns:", columns) # represents the 4 octave range in midi 36-83
        print("Total dimesions of this pianoroll is", rows*columns)

In [12]:
"""Tomb made to check if there is a single one, all zeros, or multiple ones in the pitch vector"""

def check_ones_zeros(array):
    count_ones = np.count_nonzero(array == 1)
    count_zeros = np.count_nonzero(array == 0)

    if count_ones == 1 and count_zeros == array.size - 1:
        # print("There is only one '1' and the rest are '0's in the array.")
        return True
    elif count_zeros == array.size:
        # print("All elements in the array are '0's.")
        return True
    else:
        # print("There are either multiple '1's or different values in the array.")
        return False

In [67]:
"""Read_Midi 2"""

def read_midi(filename, sop_alto, seqlen):
  
  def get_pianoroll(midi, nn_from, nn_thru, seqlen, tempo):
    pianoroll = midi.get_piano_roll(fs=2*tempo/60) # shape(128, 1262) This is the core line which makes this matrix based on 8th note
    if pianoroll.shape[1] < seqlen:
        raise UnsupportedMidiFileException

    pianoroll = pianoroll[nn_from:nn_thru, 0:seqlen] # (48, 64) Pinoroll's value still NOT binary since it has velocity
    
    binary_pianoroll = np.heaviside(pianoroll, 0) # converting as a binary matrix
    transposed_pianoroll = np.transpose(binary_pianoroll) #(64, 48)
    # transposed_pianoroll = np.transpose(pianoroll)
    # return binary_pianoroll
    return transposed_pianoroll # type numpy.ndarray

  
  # read midi file
  midi = pretty_midi.PrettyMIDI(filename)

  # An Exception error is thrown if there is a modulation(key change)
  if len(midi.key_signature_changes) !=1:
    raise UnsupportedMidiFileException

  # The Exception error thrown when tempo changes
  tempo_time, tempo = midi.get_tempo_changes()
  if len(tempo) != 1:
    raise UnsupportedMidiFileException
  if sop_alto:
    # The exception thrown if there are less than 2 parts
    if len(midi.instruments) < 2:
      raise UnsupportedMidiFileException
    # Get pianoRoll returns numpy.ndarray
    pr_s = get_pianoroll(midi.instruments[0], 36, 84, seqlen, tempo[0])
    pr_a = get_pianoroll(midi.instruments[1], 36, 84, seqlen, tempo[0])
    pr_b = get_pianoroll(midi.instruments[2], 36, 84, seqlen, tempo[0])
    wholePart = get_pianoroll(midi, 36, 84, seqlen, tempo[0])
    
    
    return pr_s, pr_a, pr_b, wholePart # All numpy.ndarray including keymode  

In [63]:
"""read generatd midi file"""
def read_melody_midi(filename, seqlen):

    def get_pianoroll(midi, nn_from, nn_thru, seqlen, tempo):
        pianoroll = midi.get_piano_roll(fs=2*tempo/60) # shape(128, 1262) This is the core line which makes this matrix based on 8th note
        if pianoroll.shape[1] < seqlen:
            raise UnsupportedMidiFileException

        pianoroll = pianoroll[nn_from:nn_thru, 0:seqlen] # (48, 64) Pinoroll's value still NOT binary since it has velocity
        
        binary_pianoroll = np.heaviside(pianoroll, 0) # converting as a binary matrix
        transposed_pianoroll = np.transpose(binary_pianoroll) #(64, 48)
        
        # print("trans pr", transposed_pianoroll)
        return transposed_pianoroll # type numpy.ndarray
    
     # read midi file
    midi = pretty_midi.PrettyMIDI(filename)

    # The Exception error thrown when tempo changes
    tempo_time, tempo = midi.get_tempo_changes()
    if len(tempo) != 1:
        raise UnsupportedMidiFileException
   
    # Get pianoRoll returns numpy.ndarray
    melody = get_pianoroll(midi.instruments[0], 36, 84, seqlen, tempo[0])
    
    return melody


In [83]:
"""Window"""

np.set_printoptions(threshold=np.inf) # Show the entire print, esp Matrix

x_all = [] # 
y_all = [] # 
files = [] # Number of files 498 but for collected files, there are 3 files that SKIPPED so we'll only see 495 files. E.g (495, int, int)
# keymodes = [] 

raw_seq_length =64
window_size = 2

try:
  if 62 < window_size or window_size < 1:
    raise WrongWindowSize(window_size)
except WrongWindowSize as e:
  print(e)  
  sys.exit()

for file in glob.glob(str(data_dir/"**/*.mid*")):
  files.append(file)
  try:
    #make a window to get sequence 
    sop_data, alt_data, bass_data, whole_data = read_midi(file, sop_alto=True, seqlen=raw_seq_length)
    
      

  # throw exception for midi data which can not be used
  except UnsupportedMidiFileException:
    print("nah")





nah
nah
nah


In [17]:
# input_data = np.array(x_all) 
# output_data = np.array(y_all)
# print(input_data.shape)
# print(output_data.shape)

# i_train, i_test = train_test_split(range(len(input_data)),test_size=int(len(input_data)/2)) 
# x_train = input_data[i_train]
# x_test = input_data[i_test] 
# print(x_train.shape)
# print(x_test.shape)

# y_train = output_data[i_train] 
# y_test = output_data[i_test] 

In [ ]:
melody_file = "/Volumes/MAGIC1/CS50/myMusicGen/data/output_KP4.mid"
melody_data = read_melody_midi(melody_file, seqlen=64)
print(check_pianoroll_dim(melody_data))

### Unused or Test code below